<a href="https://colab.research.google.com/github/saitros/100days-commit-project/blob/master/Python/Visualization/Kor_COVID_19_Visualize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KOR COVID-19 Data Visualize


After Work

-  Dataframe to Geopandas (for time series)
-  Check route decay 
   
   Use Alpha Value divided by 14

- 유동인구 : 확진자 상관계수 check (지역별, 14days)


## Install Package

In [1]:
!pip install pycountry_convert
!pip install plotly==4.6.0
!pip install folium==0.10.1

     |████████████████████████████████| 10.0MB 7.7MB/s 
     |████████████████████████████████| 235kB 49.5MB/s 
  Created wheel for pycountry: filename=pycountry-19.8.18-py2.py3-none-any.whl size=10627361 sha256=bf3015f7b2ccd9f1d0185628e8b45f027a4b29def494b5e59278a06e15d5635c
  Stored in directory: /root/.cache/pip/wheels/a2/98/bf/f0fa1c6bf8cf2cbdb750d583f84be51c2cd8272460b8b36bd3
Successfully built pycountry
ERROR: datascience 0.10.6 has requirement coverage==3.7.1, but you'll have coverage 5.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: coveralls 0.5 has requirement coverage<3.999,>=3.6, but you'll have coverage 5.1 which is incompatible.
  Found existing installation: coverage 3.7.1
    Uninstalling coverage-3.7.1:
      Successfully uninstalled coverage-3.7.1
     |████████████████████████████████| 7.2MB 2.8MB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
 

## Import Package

In [0]:
from google.colab import drive
import os

import pandas as pd
import plotly
import plotly.express as px

import folium
from folium import plugins

## Mount Colab & Path Check

In [3]:
drive.mount('/content/drive')
FOLDER = '/content/drive/My Drive/You Play AI/data/dacon'
os.chdir(FOLDER)
os.getcwd()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


'/content/drive/My Drive/You Play AI/data/dacon'

## Data Detailed Description

데이터에 관련한 설명이 따로 찾기 힘들어 데이터를 기반으로 개인적인 견해를 적었습니다

추후에 발견한 [Detailed Description](https://www.kaggle.com/kimjihoo/ds4c-what-is-this-dataset-detailed-description)



In [4]:
"""
File : Case.csv
Discussion : 불확실한 정보가 너무 많다

case_id : 확진자 고유 Key 
Province : 주거지 도/특별시 지명
City : 
group : 집단 감염 여부 True / False
Infection_case : 감염 경로(?)
Confirmed : (?)
latitude, longitude : 감염지 좌표(?)
"""

ex_df = pd.read_csv('Case.csv')
ex_df.head()

,case_id,province,city,group,infection_case,confirmed,latitude,longitude
0,1000001,Seoul,Guro-gu,True,Guro-gu Call Center,98,37.508163,126.884387
1,1000002,Seoul,Dongdaemun-gu,True,Dongan Church,20,37.592888,127.056766
2,1000003,Seoul,Guro-gu,True,Manmin Central Church,41,37.481059,126.894343
3,1000004,Seoul,Eunpyeong-gu,True,Eunpyeong St. Mary's Hospital,14,37.63369,126.9165
4,1000005,Seoul,Seongdong-gu,True,Seongdong-gu APT,13,37.55713,127.0403


In [5]:
"""
File : PatientInfo.scv
Discussion : 의미를 알 수 없는 데이터 포진

patient_id : 확진자 고유 Key
global_num : 전세계 확진자 수 None 값 많음
sex : 확진자 성별 
birth_year : 생년
age : 나이 x0 대 
country : 국적
province : 지역
city : 도시
disease : Nan or True 다른 질병 여부(?)
infection_case : 감염 경로
infection_order : 1차 감염, 2차 감염 / 감염 순서
infected_by : (?)
contact_number : 접촉자 수
symptom_onset_date : 최초 증상일 
confirmed_date : 확진일
released_date : 완치 판정일
deceased_date : 사망 판정일
state : 현 상태 4/20 
"""

ex_df = pd.read_csv('PatientInfo.csv')
ex_df.head()

,patient_id,global_num,sex,birth_year,age,country,province,city,disease,infection_case,infection_order,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state
0,1000000001,2.0,male,1964.0,50s,Korea,Seoul,Gangseo-gu,NaN,overseas inflow,1.0,NaN,75.0,2020-01-22,2020-01-23,2020-02-05,NaN,released
1,1000000002,5.0,male,1987.0,30s,Korea,Seoul,Jungnang-gu,NaN,overseas inflow,1.0,NaN,31.0,NaN,2020-01-30,2020-03-02,NaN,released
2,1000000003,6.0,male,1964.0,50s,Korea,Seoul,Jongno-gu,NaN,contact with patient,2.0,2.002000e+09,17.0,NaN,2020-01-30,2020-02-19,NaN,released
3,1000000004,7.0,male,1991.0,20s,Korea,Seoul,Mapo-gu,NaN,overseas inflow,1.0,NaN,9.0,2020-01-26,2020-01-30,2020-02-15,NaN,released
4,1000000005,9.0,female,1992.0,20s,Korea,Seoul,Seongbuk-gu,NaN,contact with patient,2.0,1.000000e+09,2.0,NaN,2020-01-31,2020-02-24,NaN,released


In [6]:
"""
File : PatientRoute.csv
Discussion : 

patient_id : 확진자 고유 Key
global_num : 전세계 확진자 수 None 값 많음
date : 경로 trace date
province : 방문지역
city : 방문 도시
type : 방문 장소
lat, long : 방문장소 위도, 경도
"""

ex_df = pd.read_csv('PatientRoute.csv')
ex_df.head()

,patient_id,global_num,date,province,city,type,latitude,longitude
0,1000000001,2.0,2020-01-22,Gyeonggi-do,Gimpo-si,airport,37.615246,126.715632
1,1000000001,2.0,2020-01-24,Seoul,Jung-gu,hospital,37.567241,127.005659
2,1000000002,5.0,2020-01-25,Seoul,Seongbuk-gu,etc,37.592560,127.017048
3,1000000002,5.0,2020-01-26,Seoul,Seongbuk-gu,store,37.591810,127.016822
4,1000000002,5.0,2020-01-26,Seoul,Seongdong-gu,public_transportation,37.563992,127.029534


In [7]:
"""
File : Weather.csv
Discussion : 

code : 지역코드
province : 지역명
date : 날짜
avg_temp : 평균기온
min_temp : 최소기온
max_temp : 최고기온
precipitation : 강수량
max_wind_speed : 최대 풍속
most_wind_direction : 풍향 
avg_relative_humidity : 평균 습도
"""

ex_df = pd.read_csv('Weather.csv')
ex_df[ex_df['date'] >= '2020-01-22'].head()

,code,province,date,avg_temp,min_temp,max_temp,precipitation,max_wind_speed,most_wind_direction,avg_relative_humidity
23711,10000,Seoul,2020-01-22,3.0,-0.1,7.8,0.0,3.2,50.0,36.0
23712,11000,Busan,2020-01-22,7.7,4.8,9.3,9.6,5.6,50.0,82.3
23713,12000,Daegu,2020-01-22,2.9,0.2,5.7,3.5,1.9,140.0,65.9
23714,13000,Gwangju,2020-01-22,5.4,3.1,6.7,9.2,4.3,50.0,82.5
23715,14000,Incheon,2020-01-22,3.0,0.0,7.2,0.0,5.3,20.0,53.4


In [8]:
"""
File : TimeProvince.csv
Discussion : 지역별 날짜에 따른 확진, 완치, 사망자 

date : 날짜 (2020-01-20 ~ )
time : 16 or 0 (측정 기준 시간(?))
province : 지역
confirmed : 확진자
released : 완치자
deceased : 사망자
"""

ex_df = pd.read_csv('TimeProvince.csv')
ex_df.tail()

,date,time,province,confirmed,released,deceased
1729,2020-04-30,0,Jeollabuk-do,18,11,0
1730,2020-04-30,0,Jeollanam-do,15,11,0
1731,2020-04-30,0,Gyeongsangbuk-do,1365,1147,52
1732,2020-04-30,0,Gyeongsangnam-do,117,97,0
1733,2020-04-30,0,Jeju-do,13,8,0


In [9]:
"""
File : TimeGender.csv
Discussion : 날짜에 따른 남여 확진자, 사망자 

date : 날짜 (2020-03-02 ~ )
time : 0 (측정 기준 시간(?))
sex : 성별
confirmed : 누적 확진자
deceased : 누적 사망자
"""

ex_df = pd.read_csv('TimeGender.csv')
ex_df.head()

,date,time,sex,confirmed,deceased
0,2020-03-02,0,male,1591,13
1,2020-03-02,0,female,2621,9
2,2020-03-03,0,male,1810,16
3,2020-03-03,0,female,3002,12
4,2020-03-04,0,male,1996,20


In [10]:
"""
File : TimeAge.csv
Discussion : 날짜에 따른 나이별 확진자, 사망자 

date : 날짜 (2020-03-02 ~ )
time : 0 (측정 기준 시간(?))
age : 나이
confirmed : 누적 확진자
deceased : 누적 사망자
"""

ex_df = pd.read_csv('TimeAge.csv')
ex_df.head(10)

,date,time,age,confirmed,deceased
0,2020-03-02,0,0s,32,0
1,2020-03-02,0,10s,169,0
2,2020-03-02,0,20s,1235,0
3,2020-03-02,0,30s,506,1
4,2020-03-02,0,40s,633,1
5,2020-03-02,0,50s,834,5
6,2020-03-02,0,60s,530,6
7,2020-03-02,0,70s,192,6
8,2020-03-02,0,80s,81,3
9,2020-03-03,0,0s,34,0


In [11]:
"""
File : Time.csv
Discussion : 날짜에 따른 검사자 반응

date : 날짜 (2020-01-20 ~ )
time : 16 or 0 (측정 기준 시간(?))
test : 피검사자수
negative : 음성
confirmed : 양성 == 확진
released : 완치
deceased : 사망
"""

ex_df = pd.read_csv('Time.csv')
ex_df.head()

,date,time,test,negative,confirmed,released,deceased
0,2020-01-20,16,1,0,1,0,0
1,2020-01-21,16,1,0,1,0,0
2,2020-01-22,16,4,3,1,0,0
3,2020-01-23,16,22,21,1,0,0
4,2020-01-24,16,27,25,2,0,0


In [12]:
"""
File : SeoulFloating.csv
Discussion : 서울 유동인구 관련 데이터

date : 날짜 (2020-01-01 ~ )
hour : 측정시간 0시 ~ 23시
birth_year : 20대, 30대, ... 70대 
sex : 성별
province : 서울시
city : 서울 구
fp_num : 유동인구 수
""" 

ex_df = pd.read_csv('SeoulFloating.csv')
ex_df.head()

,date,hour,birth_year,sex,province,city,fp_num
0,2020-01-01,0,20,female,Seoul,Dobong-gu,19140
1,2020-01-01,0,20,male,Seoul,Dobong-gu,19950
2,2020-01-01,0,20,female,Seoul,Dongdaemun-gu,25450
3,2020-01-01,0,20,male,Seoul,Dongdaemun-gu,27050
4,2020-01-01,0,20,female,Seoul,Dongjag-gu,28880


In [13]:
"""
File : SearchTrend.csv
Discussion : 출처를 알수없는 검색결과 

date : 날짜 (2016-01-01 ~ )
cold : 감기 관련 검색 (볼륨 부피 ..... 수치가 뭘 의미하는지 모르겠음)
    origin discription :  the search volume of coronavirus(flu, pneumonia, cold) in Korean language
flu : 독감 관련 검색 
pneumonia : 폐렴 관련 검색
coronavirus : 호흡기 질환 
"""

ex_df = pd.read_csv('SearchTrend.csv')
ex_df[ex_df['date'] >= '2020-01-22'].head()

,date,cold,flu,pneumonia,coronavirus
1482,2020-01-22,0.23808,0.56661,3.66416,29.74474
1483,2020-01-23,0.30308,0.55625,3.18035,100.00000
1484,2020-01-24,0.34689,0.40226,2.48156,86.11541
1485,2020-01-25,0.70888,0.39744,3.40926,62.84847
1486,2020-01-26,0.96569,0.40126,3.43989,39.62772


In [14]:
"""
File : Region.csv
Discussion : 지역관련 정보

code : 지역 코드
province : 시 / 구
city : 시 / 구 (province == city 는 각 province 의 부분합)
lat, long : 지역의 위도 경도
elementary_school_count : 초등학교 수 
kindergarten_count : 유치원 수 
university_count : 대학교 수 
academy_ratio : 학원(?) 비율
elderly_population_ratio : 고령인구 비율
elderly_alone_ratio : 독거노인 비율
nursing_home_count : 요양원 수 
"""

ex_df = pd.read_csv('Region.csv')
ex_df.head()

,code,province,city,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count
0,10000,Seoul,Seoul,37.566953,126.977977,607,830,48,1.44,15.38,5.8,22739
1,10010,Seoul,Gangnam-gu,37.518421,127.047222,33,38,0,4.18,13.17,4.3,3088
2,10020,Seoul,Gangdong-gu,37.530492,127.123837,27,32,0,1.54,14.55,5.4,1023
3,10030,Seoul,Gangbuk-gu,37.639938,127.025508,14,21,0,0.67,19.49,8.5,628
4,10040,Seoul,Gangseo-gu,37.551166,126.849506,36,56,1,1.17,14.39,5.7,1080


In [15]:
"""
File : Policy.csv
Discussion : 정책 정보
"""

ex_df = pd.read_csv('Policy.csv')
ex_df.head()

,policy_id,country,type,gov_policy,detail,start_date,end_date
0,1,Korea,Alert,Infectious Disease Alert Level,Level 1 (Blue),2020-01-03,2020-01-19
1,2,Korea,Alert,Infectious Disease Alert Level,Level 2 (Yellow),2020-01-20,2020-01-27
2,3,Korea,Alert,Infectious Disease Alert Level,Level 3 (Orange),2020-01-28,2020-02-22
3,4,Korea,Alert,Infectious Disease Alert Level,Level 4 (Red),2020-02-23,NaN
4,5,Korea,Immigration,Special Immigration Procedure,from China,2020-02-04,NaN


## Do Sth..

## With Ploty

### Plotly colors

*** About Plotly Treemap ***


for best results, ensure that the first *path* element is a single root node. In the examples below we are creating a dummy column containing identical values for each row to achieve this.


In [16]:
# Show Color Palette

fig = px.colors.sequential.swatches()
fig.show()

### main

In [17]:
df_tp = pd.read_csv('TimeProvince.csv')
df_tp.head()

,date,time,province,confirmed,released,deceased
0,2020-01-20,16,Seoul,0,0,0
1,2020-01-20,16,Busan,0,0,0
2,2020-01-20,16,Daegu,0,0,0
3,2020-01-20,16,Incheon,1,0,0
4,2020-01-20,16,Gwangju,0,0,0


In [18]:
date = df_tp['date'].max()
df_tp = df_tp[df_tp['date'] == date]
df_tp['korea'] = "South Korea"
fig = px.treemap(df_tp, path = ['korea', 'province'], values='confirmed', color='confirmed',
                 title='한국 코로나 바이러스 확진자 (4/30 기준)', color_continuous_scale='dense')
fig.show()

In [19]:
fig = px.treemap(df_tp, path = ['korea', 'province'], values='released', color='released',
                 title='한국 코로나 바이러스 완치자 (4/30 기준)', color_continuous_scale=["blue", "green", "red"])
fig.show()

In [20]:
fig = px.treemap(df_tp, path = ['korea', 'province'], values='deceased', color='deceased',
                 title='한국 코로나 바이러스 완치자 (4/30 기준)', color_continuous_scale="YlOrRd")
fig.show()

In [23]:
dt_time = pd.read_csv('Time.csv')
dt_time

,date,time,test,negative,confirmed,released,deceased
0,2020-01-20,16,1,0,1,0,0
1,2020-01-21,16,1,0,1,0,0
2,2020-01-22,16,4,3,1,0,0
3,2020-01-23,16,22,21,1,0,0
4,2020-01-24,16,27,25,2,0,0
...,...,...,...,...,...,...,...
97,2020-04-26,0,598285,578558,10728,8717,242
98,2020-04-27,0,601660,582027,10738,8764,243
99,2020-04-28,0,608514,588559,10752,8854,244
100,2020-04-29,0,614197,595129,10761,8922,246


In [21]:
df_time_age = pd.read_csv('TimeAge.csv')
df_lastest_time_age = df_time_age[df_time_age['date'] == df_time_age['date'].max()]
df_lastest_time_age['ratio'] = df_lastest_time_age['deceased'] / df_lastest_time_age['confirmed']
df_lastest_time_age

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,date,time,age,confirmed,deceased,ratio
531,2020-04-30,0,0s,140,0,0.000000
532,2020-04-30,0,10s,590,0,0.000000
533,2020-04-30,0,20s,2952,0,0.000000
534,2020-04-30,0,30s,1158,2,0.001727
535,2020-04-30,0,40s,1427,3,0.002102
536,2020-04-30,0,50s,1956,15,0.007669
537,2020-04-30,0,60s,1348,35,0.025964
538,2020-04-30,0,70s,709,74,0.104372
539,2020-04-30,0,80s,485,118,0.243299


In [24]:
df_time_age

,date,time,age,confirmed,deceased
0,2020-03-02,0,0s,32,0
1,2020-03-02,0,10s,169,0
2,2020-03-02,0,20s,1235,0
3,2020-03-02,0,30s,506,1
4,2020-03-02,0,40s,633,1
...,...,...,...,...,...
535,2020-04-30,0,40s,1427,3
536,2020-04-30,0,50s,1956,15
537,2020-04-30,0,60s,1348,35
538,2020-04-30,0,70s,709,74


In [0]:
daily_confirmed = {}
for date in set(df_time_age['date']):
    if daily_confirmed is None:
        for age in set(df_time_age['age']):
            daily_confirmed[age] = 0 #initialize
    

In [44]:
daily_confirmed

{'0s': 0,
 '10s': 0,
 '20s': 0,
 '30s': 0,
 '40s': 0,
 '50s': 0,
 '60s': 0,
 '70s': 0,
 '80s': 0}

In [36]:
df_time_age.sort_values(by=['age', 'date'])

,date,time,age,confirmed,deceased
0,2020-03-02,0,0s,32,0
9,2020-03-03,0,0s,34,0
18,2020-03-04,0,0s,34,0
27,2020-03-05,0,0s,38,0
36,2020-03-06,0,0s,45,0
...,...,...,...,...,...
503,2020-04-26,0,80s,485,115
512,2020-04-27,0,80s,485,116
521,2020-04-28,0,80s,485,116
530,2020-04-29,0,80s,485,118


## With Folium

### Plotting map with Folium

In [0]:
m = folium.Map([36, 128], zoom_start=7) 

# you can try these option 
# m = folium.Map([37, 128], zoom_start=7, tiles='Stamen Terrain') 
# m = folium.Map([37, 128], zoom_start=7, tiles='Stamen Toner') 

# with plugins Click to Fullscreen
plugins.Fullscreen(position='topright',  title='Click to Expand', 
                   title_cancel='Click to Exit', force_separate_button=True).add_to(m)

plugins.MousePosition().add_to(m)
                   
m

### Folium Visualize

In [0]:
df_patientinfo = pd.read_csv('PatientInfo.csv')
df_region = pd.read_csv('Region.csv')

In [0]:
# PatientInfo, Region 에서 필요한 부분 병합 

regional_patient = pd.merge(df_patientinfo[['patient_id','confirmed_date','sex','age','province','city']],df_region[['province','city','latitude','longitude']], how = 'left', on = ['province','city'])

In [0]:
# null Value Check 

display(regional_patient.isnull().sum() / regional_patient.shape[0])

In [0]:
"""
RecursionError: maximum recursion depth exceeded in comparison

1. data type problem --> str to float : regional_count 의 값이 str로 되어있으면 무한재귀에 빠질수있다 
        https://github.com/python-visualization/folium/issues/1105 
    Not Solved 
2. Increase maximum recursion deptyh : Python 에서는 무한재귀를 방지하기 위해 recursion maximum을 1000으로 제한하고 있다
        https://dev.to/sharmapacific/increase-maximum-recursion-depth-in-python-using-context-manager-eb4
    Not Solved
3. Version Problem --> 0.8.1 (Colab Default) to 0.10.1 (lastest Version)
    Solved
"""
regional_count = regional_patient[['latitude','longitude']].dropna() # drop missing values

m = folium.Map([36, 128], zoom_start=7, width='50%', heigth=250, title='지역별 확진자') # resize to colab 

plugins.MousePosition().add_to(m) 
plugins.MarkerCluster(regional_count).add_to(m)

m

In [0]:
male_patient = regional_patient.query('sex in ("male")')[['latitude','longitude']].dropna()
female_patient = regional_patient.query('sex in ("female")')[['latitude','longitude']].dropna()

In [0]:
m = folium.Map([36, 128], zoom_start=7, width='50%', heigth=250) 

initial = folium.plugins.MarkerCluster(control=False)

m.add_child(initial)

female_g = plugins.FeatureGroupSubGroup(initial, 'Female')
m.add_child(female_g)

male_g = plugins.FeatureGroupSubGroup(initial, 'Male') 
m.add_child(male_g)

plugins.MarkerCluster(female_patient).add_to(female_g)
plugins.MarkerCluster(male_patient).add_to(male_g)

folium.LayerControl(collapsed=False).add_to(m)

m

In [0]:
# Visualize 에는 별 의미를 두기 힘든 결과

heat_data = regional_patient.groupby(['latitude','longitude'])['patient_id'].count().reset_index()\
        .values.tolist()

m = folium.Map([36, 128], zoom_start=7, width='50%', heigth=250)#, tiles = 'stamentoner')

folium.plugins.HeatMap(heat_data).add_to(m)

m

In [0]:
df_route = pd.read_csv('PatientRoute.csv')
df_route.head()

In [0]:
'''
Issue :  Maker 사용시 Map load 가 느려짐 

1. Change Maker to MarkerCluster
    https://medium.com/@bobhaffner/folium-markerclusters-and-fastmarkerclusters-1e03b01cb7b1
    경로 표시에 적합하지 않다고 판단
        번외 : check class loader bottleneck at use diagram 
        https://user-images.githubusercontent.com/1212244/34559260-7993ee10-f141-11e7-90ad-111e0fe01bca.png


2. Dataframe to Geojson 

'''

m = folium.Map([36, 128], zoom_start=7, width='50%', heigth=250)

for id in set(df_route['patient_id']):
    point = df_route[df_route['patient_id'] == id].loc[:, 'latitude':'longitude'].values
    
    for each_point in point:
        folium.Marker(each_point).add_to(m)


folium.PolyLine().add_to(m) 

m